# Dask - Exploratory Data Analysis

In this courselet, we will explore the use of Dask as a Data Science tool when working with Big Data. By the end of this courselet, you would be able to:

- Identify Sparks main objects
- Identify some techniques to optimize Dask's performance.

## What is Dask?

[Dask](https://www.dask.org/) is an open-source Python library designed for parallel computing. It is considered to be an optimal framework to scale python projects by allowing the integration of distributed clusters. 

## Advantages of Dask

- Purely pythonic: Dask is completely written in Python, and as such, it facilitates the interaction with other popular data science and machine learning libraries like Numpy, Pandas, scikit-learn, PyTorch, TensorFlow, Keras, among others.
- User friendly: Given that dask is purely pythonic, its adoption, maintenance, development and debugging can be considered more user friendly compared to Spark.

## Disadvantages of Dask

- Large data limitations: Although Dask is a framework designed to deal with large datasets by relying in distributed and parallel computing, it might not be the best solution for cases of very large data (let's say TB's of serial data or streaming data). In those cases, Spark can be considered a more appropiate solution.
- Limited fault tolerance: Although Dask offers some fault tolerance mechanisms within its design, these are not as complex nor effective as the ones offered by Spark. 

## Connecting to our cluster

The first thing that we are going to do, is to connect to our Dask cluster. We are going to create a [Client](https://distributed.dask.org/en/latest/client.html) instance to connect to the scheduler and be able to send jobs to the workers. 

**NOTE:** For local testing and practice, review the documentation to set up a [LocalCluster](https://docs.dask.org/en/stable/deploying-python.html)

In [ ]:
import dask
from dask.distributed import Client, LocalCluster

client = Client("tcp://dask-scheduler:8786")
client

## Task Graph

A main concept to understand Dask is the concept of [Task Graph](https://docs.dask.org/en/latest/graphs.html). A Task Graph is a representation of the execution plan that Dask defines for the requested jobs. In this plan, a node represents a task, while the edges represent the dependencies between tasks. 

Similar to Spark, Dask works under a [Lazy Evaluation](https://saturncloud.io/blog/a-data-scientist-s-guide-to-lazy-evaluation-with-dask/) principle, which means that we can define as many execution tasks as desired within an execution plan, but non of them is executed until explicitly told to do so. This allows Dask to find an optimal execution plan by parallelizing the computation and avoiding reduntant processes. 

## Dask Array

The first data structure that we are going to work with, is the [Dask Array](https://docs.dask.org/en/latest/array.html) A Dask Array is essentially a collection of Numpy arrays distributed among the cores of the system, to allow for parallelization. This distribution and coordination is organized through the use of graphs. 

When defining a new array, a key concept to keep in mind is the concept of [Chunks](https://docs.dask.org/en/latest/array-chunks.html) A chunk represents each of the different arrays or pieces that compose the whole Dask Array and that are distributed along the cores. [The choice of the right number of chunks](https://blog.dask.org/2021/11/02/choosing-dask-chunk-sizes) entirely depends on the algorithm that we are going to execute and the size of our data. 

Now that we have covered a couple of relevant concepts in Dask, let's start running a first example. In the following lines of code, we are going to do the following:

- Define a simple [Dask Array](https://docs.dask.org/en/latest/array.html)
- Define the calculation of the sin of each element in the array
- Define the calculation of the mean of all the elements within the array
- Visualize the Task Graph to get a better understanding on how the job is executed
- Compute the whole process

In [ ]:
import dask.array as da
rng = da.random.default_rng()
x = rng.random((4, 4), chunks=(2, 2)) # We create a 4x4 array, splitted into chunks of size 2x2. Therefore, we will have 4 chunks (4x4/2x2 = 4)
y = da.sin(x)
mean = y.mean()

In [ ]:
# Now, let's look at the Task Graph

mean.visualize()

As we see, the graph is representing the execution plan, in which we have 4 chunks of random values from our array. Each chunk is then passed to calculate the sin for each value, then the mean is calculated for each chunk and finally means are aggregated to return the final result.

Now that we have our plan, we can execute it through the *.compute()* method. 

In [ ]:
mean.compute()

## Custom Delayed Plans

Dask possesses the flexibility of allowing us to define our own *lazy algorithm* by making use of the [delayed](https://docs.dask.org/en/stable/delayed.html) interface. This is mostly helpful for cases in which parallelization is achievable, but the structure of our data cannot be abstracted as an array or dataframe. The following example taken from the official documentation exemplifies a case clearly:

In [ ]:
# Suppose we have the following functions and loop:
def inc(x):
    return x + 1

def double(x):
    return x * 2

def add(x, y):
    return x + y

data = [1, 2, 3]

output = []
for x in data:
    a = inc(x)
    b = double(x)
    c = add(a, b)
    output.append(c)

total = sum(output)
total

The previous code was eagerly executed. However, there was certainly room for an execution plan that could parallelize the tasks. For example, instead of actually looping through the elements of the list and performing the computations sequentially, a plan could take each element of the list and perform both the addition and the doubling for all elements in parallel, before adding the pairs and finally summing up all of the values.

This is achievable through the [delayed](https://tutorial.dask.org/03_dask.delayed.html) method:

In [ ]:
output = []
for x in data:
    a = dask.delayed(inc)(x)
    b = dask.delayed(double)(x)
    c = dask.delayed(add)(a, b)
    output.append(c)

total_delayed = dask.delayed(sum)(output)

In [ ]:
# Let's visualize our graph
total_delayed.visualize()

Now the initial code is represented as a graph. We can now perform the computation:

In [ ]:
total_delayed.compute()

We can also pre-define our functions to be executed lazily through the *@delayed* decorator:

In [ ]:
@dask.delayed
def inc_del(x):
    return x + 1

@dask.delayed
def double_del(x):
    return x * 2

@dask.delayed
def add_del(x, y):
    return x + y

output = []
for x in data:
    a = inc_del(x)
    b = double_del(x)
    c = add_del(a, b)
    output.append(c)

total_dec = dask.delayed(sum)(output)

In [ ]:
total_dec.compute()

## The Dask DataFrame

The main object to work with a dataset through Dask is the [Dask DataFrame](https://docs.dask.org/en/latest/dataframe.html), which is essentially a collection of multiple Pandas DataFrames parallelized across the cluster. 

To begin our exploratory data analysis, we'll start by loading the data and creating our dataframe. For this courselet, we are using taxi trips reported to the City of Chicago in 2020. This data is publicly available through the [Chicago Data Portal](https://data.cityofchicago.org/en/Transportation/Taxi-Trips-2020/r2u4-wwk3/about_data). 

For this example, we are using the data in [Parquet format](https://parquet.apache.org/) This open-source format offers efficient data storage and compression, as well as better data retrieval than traditional CSV. Therefore, given the data is available in this format, we take advantage of it.

In [ ]:
# Loading the df
import dask.dataframe as dd

ddf = dd.read_parquet("data/chicago-taxi-2020.parquet", assume_missing=True)
ddf.head()

We will now start performing some descriptive analysis.

In [ ]:
# Counting non-empty rows per column
non_empty_count = ddf.count()
non_empty_count

Why we cannot see the results? As we stated at the beginning, Dask will not perform any computation until explicitly calling the *.compute()* method.

In [ ]:
# Now we will get the actual result
non_empty_count.compute()

In [ ]:
# Missingness rate per column
ddf.isnull().mean().compute()

In [ ]:
# Showing the top 10 rows of 2 columns
ddf[['Trip ID', 'Trip Total']].head(10)

In [ ]:
# Mean of a column
ddf['Trip Total'].mean().compute()

In [ ]:
# Max value of a column
ddf['Trip Total'].max().compute()

## Visualizing Data

Given that Dask is a completely pythonic framework, data visualization is supported through popular libraries such as matplotlib and seaborn. Let's do a quick visualization example:

In [ ]:
import matplotlib.pyplot as plt

# Let's say we want to visualize the avg trip total cost by payment type
viz_ddf = ddf.dropna(subset=['Trip Total', 'Payment Type']) # We subset the data
avg_trip_total_by_payment_type = viz_ddf.groupby('Payment Type')['Trip Total'].mean().compute() # We create a small pd.df with the information grouped

# Now we visualize as we would normally do
avg_trip_total_by_payment_type.plot(kind='bar', color='skyblue')
plt.xlabel('Payment Type')
plt.ylabel('Average Trip Total (USD)')
plt.title('Average Trip Total by Payment Type')
plt.xticks(rotation=45)
plt.tight_layout()  
plt.show()

More on data visualization can be found in the [FOUNT Courselets library](https://voices.uchicago.edu/fount/resources/fount-courselets/).

## Improving Dataframes performance

When working with dataframes, Dask suggests a series of techniques that can help us improve the performance. Let's take a look at some of them.

**Repartition** 

When we load a dataframe, Dask will evaluate the resources, datasize and type and choose an n number to partition and distribute our dataframe. However, this n number of partitions might not be the most optimal, and so, we can perform a repartition to our data to potentially improve performance. 

In [ ]:
# First, let's take a look at the current number of partitions
ddf.npartitions

In [ ]:
import time

def measure_time(file_path, num_partitions):
    # Read the data and repartition
    ddf = dd.read_parquet(file_path, assume_missing=True)
    ddf = ddf.repartition(npartitions=num_partitions)
    
    # Measure computation time
    start_time = time.time()
    result = ddf['Trip Total'].mean().compute()
    end_time = time.time()
    
    return end_time - start_time

file_path = "data/chicago-taxi-2020.parquet"

# Different numbers of partitions to test
num_partitions_list = [1, 2, 4, 8, 16]

times = []
for num_partitions in num_partitions_list:
    time_taken = measure_time(file_path, num_partitions)
    times.append(time_taken)

# Plotting the results
plt.plot(num_partitions_list, times, marker='o')
plt.xlabel('Number of Partitions')
plt.ylabel('Time taken (seconds)')
plt.title('Impact of Number of Partitions on Computation Time')
plt.xticks(num_partitions_list)
plt.grid(True)
plt.show()

# Identify the Number of Partitions with the Lowest Time
min_time = min(times)
optimal_num_partitions = num_partitions_list[times.index(min_time)]
print(f"The optimal number of partitions is {optimal_num_partitions} with a time of {min_time} seconds")

**Indexing a column**

Indexing our dataframe by a column through the use of the [*.set_index()*](https://docs.dask.org/en/latest/generated/dask.dataframe.DataFrame.set_index.html) method can help us perform faster and more efficient queries. In this case, we can take advantage of one of the two timestamps and create an index from it. This would be a smart strategy as this data is very likely to be queried constantly incorporating temporality conditions.

In [ ]:
# We start by generating a new column in time format. We make this choice instead of overwriting the original column
ddf['timestamp_index'] = dd.to_datetime(ddf['Trip Start Timestamp']) 

# Now we make index our ddf using the new column
ddf = ddf.set_index('timestamp_index', sorted=True)

In [ ]:
# Now, we can use our new index to subset our data
january_ddf = ddf.loc['2020-01'] # This operation is much faster through an index than without it

january_ddf.head(10)

**Persisting Data**

Another useful technique to improve our performance is to [persist](https://docs.dask.org/en/stable/api.html#dask.persist) in memory the recurrently used data. By doing this, we avoid the constant recomputation of the dataset, and our queries are executed atop of a persisted object, optimizing performance. According to Dask documentaion, the following would be a smart workflow:

1. Load the data
2. Filter (if needed) the data to a particular subset
3. Set a smart index
4. Persist the data in memory with the use of the *client.persist* method.

We can do a example of this workflow by using the examples we have shown previously:

In [ ]:
# 1. Data Loading
ddf = dd.read_parquet("data/chicago-taxi-2020.parquet", assume_missing=True) 

# 2. Filtering - Let's say we only want to analyze the trips that were payed with Cash
cash_ddf = ddf[ddf["Payment Type"] == "Cash"]

# 3. Indexing - We use our previous timestamp example
cash_ddf['timestamp_index'] = dd.to_datetime(cash_ddf['Trip Start Timestamp']) 
cash_ddf = cash_ddf.set_index('timestamp_index', sorted=True)

# 4. Persisting data
cash_ddf = client.persist(cash_ddf)

In [ ]:
# Now we can perform operations which should be faster
cash_ddf['Trip Total'].mean().compute()

This section has been developed by following [official documentation on Best Practices](https://docs.dask.org/en/stable/dataframe-best-practices.html#).

## Bags

The final type of Dask object that we are going to explore are [Dask Bags](https://docs.dask.org/en/stable/bag.html) Dask Bags are a special type of object that allows us to perform parallelized operations on Python objects that do not follow the structure of a Pandas Dataframe nor Numpy Array. Bags are particullarly useful when working with unstructured data like JSON files. Lets look at a example.

In [ ]:
# Imagine that we had a json file that looked like the following unstructured data, representing a series of social media posts

posts = [
    {"id": 1, "text": "Love using #dask for data processing!", "tags": ["data", "python", "dask"], "user": {"id": 100, "name": "John"}},
    {"id": 2, "text": "Exploring the #python ecosystem is fascinating. #datascience", "user": {"id": 101, "name": "Jane"}},
    {"id": 3, "text": "Check out my latest #blog on #machinelearning.", "tags": ["AI", "blog", "machinelearning"], "user": {"id": 102, "name": "Jack"}},
    # Some posts might not have hashtags or are missing other fields
    {"id": 4, "text": "Just another day training a #machinelearning model with #dask.", "user": {"id": 103, "name": "Jill"}}
]


As you can observe, the *posts* data is not structured, as some posts come with tags and some other don't. Imagine an scenario in which we wanted to analyze thousands of posts from a social media site. We would certainly like to take advantage of parallelization to do this process. This is where Dask Bags can help us. Suppose that we wanted to count the frequency of hashtags among posts. The following code performs that operation by transforming our posts data into a bag:

In [ ]:
import dask.bag as db
import re

posts_bag = db.from_sequence(posts)

# Function to extract hashtags from text
def extract_hashtags(post):
    hashtags = re.findall(r"#(\w+)", post["text"])
    return hashtags

# Function to filter posts with text field
def filter_posts_with_text(post):
    return "text" in post

# Apply filter and map functions, then flatten the list of lists
hashtags = posts_bag.filter(filter_posts_with_text).map(extract_hashtags).flatten()

# Count occurrences of each hashtag
hashtag_counts = hashtags.frequencies().compute()

hashtag_counts
